In [1]:
import copy

In [2]:
data = '''x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj'''

In [3]:
with open('data/12-24.input') as f:
    data = f.read().strip()

## Part I

In [4]:
start = data.split('\n\n')[0].split('\n')

g = data.split('\n\n')[1].split('\n')

gates = []
for line in g:
    t = line.split(' -> ')
    inputs = t[0].split(' ')
    gates.append([(inputs[0], inputs[2]), (inputs[1], t[1])])

wires = []
for x in gates:
    wires.extend([x[0][0], x[0][1], x[1][1]])

wires = set(wires)

#  Initialize the values of the inputs, None if there is no input,
#  otherwise 0 or 1.  
numbers = { x: None for x in wires }
for line in start:
    t = line.split(': ')
    numbers[t[0]] = int(t[1])

In [5]:
save_gates = copy.deepcopy(gates)

The main idea is just to push through the values in the network.  We don't know the exact order in which to do that, so we check if the inputs are not `None` and, if so, evaluate and store the result in the proper key in the dictionary.  Continue until all of the values are not `None`.  

In [6]:
def run(gates, numbers):
    while any(x is None for x in numbers.values()):
        unused = []
        for i in gates:
            k = i[0]
            v = i[1]
            if numbers[k[0]] is not None and numbers[k[1]] is not None:
                match v[0]:
                    case 'AND':
                        numbers[v[1]] = numbers[k[0]] & numbers[k[1]]
                    case 'OR':
                        numbers[v[1]] = numbers[k[0]] | numbers[k[1]]
                    case 'XOR':
                        numbers[v[1]] = numbers[k[0]] ^ numbers[k[1]]
                    case _:
                        print('Oops')
            else:
                unused.append(i)
        gates = unused
        
    final_keys = sorted([x for x in numbers.keys() if x.startswith('z')], reverse=True)
    s = int(''.join(str(numbers[k]) for k in final_keys), base=2)
    return s

In [7]:
run(gates, numbers)

51107420031718

In [8]:
gates == save_gates

True

## Part II

In [9]:
print(f'Number of x inputs: {sum(k.startswith("x") for k in numbers.keys())}')
print(f'Number of y inputs: {sum(k.startswith("y") for k in numbers.keys())}')
print(f'Number of z outputs: {sum(k.startswith("z") for k in numbers.keys())}')

Number of x inputs: 45
Number of y inputs: 45
Number of z outputs: 46


In [10]:
def pad(number, size=2):
    s = str(number)
    if len(s) < size:
        s = '0'*(size - len(s)) + s
    return s

In [11]:
def find_parents(target, gates):
    parents = []
    for g in gates:
        if g[1][1] == target:
            parents.append(g)
    return parents

Each final output gate $\{z_0, \ldots, z_{45}\}$ needs to be the XOR of two inputs (to see if exactly one input is 1, or if there is a carry for the next higher significant digit).  Can we find output gates that don't satisfy this condition?  Then we have found some of the outputs that need to be switched to correct the adder to work correctly.

In [12]:
for i in range(46):
    parents = find_parents('z'+pad(i), gates)
    if len(parents) != 1:
        print(pad(i))
    elif parents[0][1][0] != 'XOR':
        print(pad(i), parents)

10 [[('x10', 'y10'), ('AND', 'z10')]]
21 [[('ptd', 'scj'), ('AND', 'z21')]]
33 [[('jtg', 'trf'), ('OR', 'z33')]]
45 [[('wtc', 'ndp'), ('OR', 'z45')]]


So, let's test the inputs of the form `000...0111` and `000...000` to see if the "adder" works correctly, starting from the lowest number.

In [13]:
def test(x_str, y_str, numbers, gates):
    for k in numbers.keys():
        if not (k.startswith('x') or k.startswith('y')):
            numbers[k] = None
    x_keys = sorted([k for k in numbers.keys() if k.startswith('x')], reverse=True)
    y_keys = sorted([k for k in numbers.keys() if k.startswith('y')], reverse=True)
    for k, v in zip(x_keys, list(x_str)):
        numbers[k] = int(v)
    for k, v in zip(y_keys, list(y_str)):
        numbers[k] = int(v)
        
    network_output = str(bin(run(gates, numbers))[2:])
    network_output = '0'*(46-len(network_output)) + network_output
    true_output = str(bin(int(x_str, base=2) + int(y_str, base=2))[2:])
    true_output = '0'*(46 - len(true_output)) + true_output
    return ' ' + x_str, ' ' + y_str, true_output, network_output

Let's first test up to the first output that doesn't work correctly, suspecting that it will be switched with an input that is "lower" than itself.  

In [14]:
upper = 12

for i in range(upper):
    x = '0'*(45-i) + '1'*i
    y = '0'*45
    result = test(x, y, numbers, gates)
    if result[2] != result[3]:
        print(f'{result[0]}\n{result[1]}\n{result[2]}\n{result[3]}')
        print('------')

 000000000000000000000000000000000011111111111
 000000000000000000000000000000000000000000000
0000000000000000000000000000000000011111111111
0000000000000000000000000000000000101111111111
------
